## PICMUS beamforming with usbmd toolbox
The PICMUS dataset is one of the most widely used ultrasound datasets in the research community. The dataset is available at https://www.creatis.insa-lyon.fr/Challenge/IEEE_IUS_2016/.
In this tutorial we show you how to use the dataset with the usbmd toolbox.

First make sure you have installed the usbmd toolbox using our [installation instructions](../../README.md). Next, we need to prepare the PICMUS dataset for use with the toolbox. After download, convert the dataset to the usbmd format using the following command:
```bash
python usbmd/data_format/convert_picmus.py --source <path_to_downloaded_dataset> --output <path_to_output_dir>
```

Also make sure to then correctly setup your data paths by running
 ```bash
 python usbmd/common.py
 ```
All paths in the repository are relative to the root of the repository you set using the previous step.

Then add the (relative) location of your converted PICMUS dataset to `./configs/config_picmus_rf.yaml`. In this tutorial, we assume the following relative path:

```yaml
# ./configs/config_picmus_rf.yaml
data:
  dataset_folder: "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
```


In [ ]:
# Change the working directory to the root of the project by moving up in the
# directory tree until the file 'setup.py' is found.
import os

while not os.path.exists('setup.py'):
    os.chdir('..')

# Set to True to run the notebook quickly for testing purposes This is done
# automatically in 'tests/test_run_notebooks.py'
quick_mode = True

In [ ]:
# injected parameters
quick_mode = False

In [ ]:
from usbmd.ui import DataLoaderUI
from usbmd.setup_usbmd import setup
from usbmd.processing import rf2iq, downsample, complex_to_channels

### PICMUS

### Experiments - resolution distortion

#### RF beamforming
Let's load the RF data and beamform it. First have a look at the most easy way to do it using our `DataloaderUI` class and setup config functions.

In [ ]:
# Loads config, sets data paths and initializes gpu if available
config = setup(config_path="./configs/config_picmus_rf.yaml")

# Override config settings
config.plot.save = False

# data paths
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
config.data.file_path = "contrast_speckle_simu_dataset_rf.hdf5"

In [ ]:
if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)

#### IQ beamforming
We can also load the IQ data and beamform it.

No need to reload a new config (although you can use `./configs/config_picmus_iq.yaml`).
We can simply set n_ch to 2 and change the data file path

In [ ]:
config.scan.n_ch = 2
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_iq"
config.data.file_path = "contrast_speckle_simu_dataset_iq.hdf5"

In [ ]:
if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)

#### IQ beamforming, but reading RF data
For this, we need a more manual approach. Let's first load the RF data, but now in `raw_data` format. After that we can manually convert it to IQ data and beamform it, both using our `Process` class.

In [ ]:
# data paths
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
config.data.file_path = "contrast_speckle_simu_dataset_rf.hdf5"
config.scan.n_ch = 1
config.scan.downsample = 4

if not quick_mode:
    ui = DataLoaderUI(config)
    # >> load data
    raw_data_rf = ui.get_data()

    # >> convert RF to IQ
    print(f"RF data has shape {raw_data_rf.shape}")
    raw_data_iq = rf2iq(raw_data_rf, "hilbert")
    raw_data_iq = downsample(raw_data_iq, config.scan.downsample, axis=1)
    # data needs to be 2 channels instead of complex form for other processing functions
    raw_data_iq = complex_to_channels(raw_data_iq.squeeze(axis=-1), axis=-1)
    print(f"IQ data has shape {raw_data_iq.shape}")

    # >> beamform, for now we need to reinitialize the ui such that the beamformer is reinitialized for IQ
    config.scan.n_ch = 2
    ui = DataLoaderUI(config)
    image = ui.process.run(raw_data_iq, dtype="raw_data", to_dtype="image")
    ui.dtype = "image"
    ui.data = image

    # >> plot image
    ui.plot(image)

### Simulations - resolution distortion
For fun, let's look at another PICMUS dataset as an example. This is a simulated dataset, generating a bunch of scatterers. Often we can use it to look at the resolution of our ultrasound pipeline.

#### RF beamforming

In [ ]:
# data paths
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_rf"
config.data.file_path = "resolution_distorsion_simu_dataset_rf.hdf5"
config.scan.n_ch = 1

if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run()

The quality is not really impressive. But wait, in the config we specified the number of transmits (in this case number of plane wave angels) to be 7. The total in the dataset is 75. Let's see how it looks using all of them. 
This can take a bit longer... Reduce the number of transmits or use cpu instead of gpu if you run into any issues.

In [ ]:
if not quick_mode:
    config.scan.selected_transmits = 30
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)